In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
nx, ny = 32, 32 # number of computational grids
dx, dy = 0.5e-6, 0.5e-6 # spacing of computational grid [m]
eee = 1.0e+6 # driving force of growth of phase B: g_A - g_B [J/m3]
gamma = 1.0 # ineterfacial energy [J/m2]
delta = 4.*dx # interfacial thickness [m]
amobi = 4.e-14 # interfacial mobilitiy [m4/(Js)]
ram = 0.1 # paraneter which deternines the interfacial area
bbb = 2.*np.log((1.+(1.-2.*ram))/(1.-(1.-2.*ram)))/2.  # The constant b = 2.1972

In [ ]:
aaa   = np.sqrt(3.*delta*gamma/bbb) # gradient energy coefficient "a" [(J/m)^(1/2)]
www   = 6.*gamma*bbb/delta # potential height W [J/m3]
pmobi = amobi*np.sqrt(2.*www)/(6.*aaa) # mobility of phase-field [m3/(Js)]

In [ ]:
dt = 0.1 # time increment for a time step [s]
nsteps = 1000 # total number of time step

In [ ]:
p  = np.zeros((nx,ny)) # phase-field variable
dfdp = np.zeros([nx, ny])

In [ ]:
r_nuclei = 5.*dx # radius of the initial B phase
for i in range(nx):
    for j in range(ny):
        r = np.sqrt( ((i-nx/2) *dx)**2 +((j-ny/2) * dy)**2 ) - r_nuclei
        p[i,j] = 0.5*(1.-np.tanh(np.sqrt(2.*www)/(2.*aaa)*r))

In [ ]:
def calc_wave_vector(nx, ny, dx, dy):
    half_nx = int(nx/2)
    half_ny = int(ny/2)
    dkx = (2.0 * np.pi) / (nx * dx)
    dky = (2.0 * np.pi) / (ny * dy)
    k2 = np.zeros([nx, ny])
    
    for i in range(nx):
      if i < half_nx:
        kx = i*dkx
      else:
        kx = (i-nx)*dkx
      kx2 = kx**2

      for j in range(ny):
        if j < half_ny:
          ky = j*dky
        else:
          ky = (j-ny)*dky
        ky2 = ky**2

        k2[i,j] = kx2 + ky2       
    return k2

In [ ]:
k2 = calc_wave_vector(nx, ny, dx, dy)

In [ ]:
for istep in range(1, nsteps):

  pk = np.fft.fftn(p)
  dfdp[:,:] = 4.0 * www * p[:,:]*(1 - p[:,:])*(p[:,:] - 0.5 + 3.0/2.0/www * eee) 

  dfdpk = np.fft.fftn(dfdp)
 
  nummer = pmobi * dfdpk[:,:] * dt
  denom = 1.0 + pmobi * k2[:,:] * aaa*aaa * dt
  pk[:,:] = (pk[:,:] + nummer) / denom

  p = np.real(np.fft.ifftn(pk))

  for i in range(1, nx):
    for j in range(1, ny):
      if(p[i, j] >= 0.9999):
        p[i, j] = 0.9999
      if(p[i, j] < 0.00001):
        p[i, j] = 0.00001

  if istep % 100 == 0:
    print('nstep = ', istep)
    plt.imshow(p, cmap='bwr')
    plt.title('phase-field')
    plt.colorbar()
    plt.show() 